#### 0. Base 작업
아래 awk 리눅스 명령어 코드로 필요한 Respiratory 진단 받은 환자의 필요한 데이터만 추출<br>
awk -F, 'FNR==NR{a[$1,$2]=1;next} ($2,$3) in a{print $2 "," $3 "," $5 "," $6 "," $10} ($2,$3) in a && NF==14{$14=""; print}' project/MIMIC-III/Data/SUBJECTID.csv /data/PUBLIC_DATA/MIMIC-III/CHARTEVENTS.csv | awk 'NR == 1 {print "SUBJECT_ID,HADM_ID,CHARTTIME,ITEMID,VALUE"} NR > 1 {print}' > project/MIMIC-III/Data/id_value.csv

In [17]:
# # 첫번째 row를 column명으로 사용하기
# df = pd.read_csv("~/project/MIMIC-III/Data/id_value.csv")
# df = df.rename(columns={'CHARTTIME': 'ITEMID', 'ITEMID':'CHARTTIME', 'VALUE':'VALUENUM'})
# df.to_csv("~/project/MIMIC-III/Data/id_value.csv", index=False)
# df

In [ ]:
import pandas as pd
import numpy as np

chartevents = pd.read_csv("~/project/MIMIC-III/Data/id_value.csv")
chartevents

In [42]:
# ITEMID 
d_item = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ITEMS.csv")
d_item = d_item[['ITEMID', 'LABEL']]
d_item

,ITEMID,LABEL
0,497,Patient controlled analgesia (PCA) [Inject]
1,498,PCA Lockout (Min)
2,499,PCA Medication
3,500,PCA Total Dose
4,501,PCV Exh Vt (Obser)
...,...,...
12482,226757,GCSMotorApacheIIValue
12483,226758,GCSVerbalApacheIIValue
12484,226759,HCO3ApacheIIValue
12485,226760,HCO3Score


#### 1. CHARTEVENTS table 생성

##### 1-1. 24h 이내 데이터만 남기기

In [49]:
df = chartevents

# 결측치 제거
df.dropna()
# value에 들어간 문자 데이터 제거
df = df[pd.to_numeric(df['VALUENUM'], errors='coerce').notnull()]

# 환자별 첫 24시간 이내의 데이터만 남기기
df['CHARTTIME'] = pd.to_datetime(df['CHARTTIME']) # CHARTTIME 컬럼을 datetime 자료형으로 변환

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False).apply(lambda x: x[(x['CHARTTIME'] == x['CHARTTIME'].min()) | ((x['CHARTTIME'] > x['CHARTTIME'].min()) & (x['CHARTTIME'] <= x['CHARTTIME'].min() + pd.Timedelta(hours=24)))])
df1 = df1.reset_index(drop=True)

df1

/tmp/ipykernel_3275482/1674232857.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CHARTTIME'] = pd.to_datetime(df['CHARTTIME']) # CHARTTIME 컬럼을 datetime 자료형으로 변환


,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUENUM
0,36,165660,223835,2134-05-12 12:00:00,100.0
1,36,165660,224328,2134-05-12 12:00:00,0.37
2,36,165660,224329,2134-05-12 12:00:00,6.0
3,36,165660,224330,2134-05-12 12:00:00,2.5
4,36,165660,224331,2134-05-12 12:00:00,0.0
...,...,...,...,...,...
5027422,99985,176670,224055,2181-01-30 00:00:00,3
5027423,99985,176670,224056,2181-01-30 00:00:00,1
5027424,99985,176670,224057,2181-01-30 00:00:00,3
5027425,99985,176670,224058,2181-01-30 00:00:00,1


In [50]:
# 환자별 첫 24시간 이내의 데이터의 평균 구하기
df1.fillna(0, inplace=True)  # 결측치를 0으로 변경
df2 = df1.groupby(['SUBJECT_ID', 'HADM_ID', 'ITEMID'], as_index=False).agg({'VALUENUM': lambda x: np.mean(x.astype(float))})
df2

,SUBJECT_ID,HADM_ID,ITEMID,VALUENUM
0,36,165660,220045,81.961538
1,36,165660,220046,113.333333
2,36,165660,220047,50.000000
3,36,165660,220179,108.041667
4,36,165660,220180,62.166667
...,...,...,...,...
823687,99985,176670,227466,47.700000
823688,99985,176670,227467,1.100000
823689,99985,176670,227565,2200.000000
823690,99985,176670,227566,2800.000000


In [51]:
# column으로 추가될 unique ITEMID 개수 확인
unique_id = df2['ITEMID'].unique()
len(unique_id)

1329

In [52]:
# 24h laboratory value의 pivot table 생성
pivot_table = df2.pivot_table(index=['SUBJECT_ID', 'HADM_ID'], columns='ITEMID', values='VALUENUM', aggfunc='first')
pivot_table = pivot_table.fillna(0)
df_pivot = pivot_table.reset_index(level=['SUBJECT_ID', 'HADM_ID'])
chartevent_value_24h = df_pivot
chartevent_value_24h

ITEMID,SUBJECT_ID,HADM_ID,2,3,4,5,24,25,26,29,...,228381,228382,228394,228395,228396,228409,228410,228411,228412,228444
0,36,165660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,91,121205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,94,140037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101,175533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,111,192123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7540,99897,181057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,2.0,0.0,5.0,5.0,5.0,5.0,0.0
7541,99899,188409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7542,99912,189380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7543,99939,159023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
# 칼럼 별로 관측되지 않은 laboratory value 값(0) 비율 확인 
zero_ratios = (chartevent_value_24h == 0).sum() / len(chartevent_value_24h)
zero_ratios = zero_ratios.to_frame()[2:]
zero_ratios = zero_ratios.reset_index().rename(columns={'index': 'ITEMID', 0:'ZERO_RATIOS'})
zero_ratios = pd.merge(zero_ratios, d_item[["ITEMID", "LABEL"]], on='ITEMID')
zero_ratios = zero_ratios.reindex(columns=["ITEMID", "LABEL", "ZERO_RATIOS"])
zero_ratios = zero_ratios.sort_values(by='ZERO_RATIOS', ascending=True)
zero_ratios

,ITEMID,LABEL,ZERO_RATIOS
1012,226543,Religion,0.468125
647,220602,Chloride (serum),0.481246
1154,227442,Potassium (serum),0.481246
649,220615,Creatinine,0.481378
655,220645,Sodium (serum),0.481511
...,...,...,...
1111,227063,WBCScore_ApacheIV,1.000000
1052,226775,SodiumApacheIIScore,1.000000
1109,227061,Ventilated at any time during ICU Day 1,1.000000
1054,226777,TempApacheIIScore,1.000000


In [55]:
# chartevent_value_24h.to_csv("~/project/MIMIC-III/Data/CHARTEVENT_VALUE_24h.csv", index=False)

##### 1-2. 48h 이내 데이터만 남기기

In [56]:
df3 = chartevents

# 결측치 제거
df3.dropna()
# value에 들어간 문자 데이터 제거
df3 = df3[pd.to_numeric(df3['VALUENUM'], errors='coerce').notnull()]

# 환자별 첫 24시간 이내의 데이터만 남기기
df3['CHARTTIME'] = pd.to_datetime(df3['CHARTTIME']) # CHARTTIME 컬럼을 datetime 자료형으로 변환

df4 = df3.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False).apply(lambda x: x[(x['CHARTTIME'] == x['CHARTTIME'].min()) | ((x['CHARTTIME'] > x['CHARTTIME'].min()) & (x['CHARTTIME'] <= x['CHARTTIME'].min() + pd.Timedelta(hours=24)))])
df4 = df4.reset_index(drop=True)

df4

/tmp/ipykernel_3275482/3957932851.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['CHARTTIME'] = pd.to_datetime(df3['CHARTTIME']) # CHARTTIME 컬럼을 datetime 자료형으로 변환


,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUENUM
0,36,165660,223835,2134-05-12 12:00:00,100.0
1,36,165660,224328,2134-05-12 12:00:00,0.37
2,36,165660,224329,2134-05-12 12:00:00,6.0
3,36,165660,224330,2134-05-12 12:00:00,2.5
4,36,165660,224331,2134-05-12 12:00:00,0.0
...,...,...,...,...,...
5027422,99985,176670,224055,2181-01-30 00:00:00,3
5027423,99985,176670,224056,2181-01-30 00:00:00,1
5027424,99985,176670,224057,2181-01-30 00:00:00,3
5027425,99985,176670,224058,2181-01-30 00:00:00,1


In [57]:
# 환자별 첫 48시간 이내의 데이터의 평균 구하기
df4.fillna(0, inplace=True)  # 결측치를 0으로 변경
df5 = df4.groupby(['SUBJECT_ID', 'HADM_ID', 'ITEMID'], as_index=False).agg({'VALUENUM': lambda x: np.mean(x.astype(float))})
df5

,SUBJECT_ID,HADM_ID,ITEMID,VALUENUM
0,36,165660,220045,81.961538
1,36,165660,220046,113.333333
2,36,165660,220047,50.000000
3,36,165660,220179,108.041667
4,36,165660,220180,62.166667
...,...,...,...,...
823687,99985,176670,227466,47.700000
823688,99985,176670,227467,1.100000
823689,99985,176670,227565,2200.000000
823690,99985,176670,227566,2800.000000


In [58]:
# column으로 추가될 unique ITEMID 개수 확인
unique_id = df5['ITEMID'].unique()
len(unique_id)

1329

In [59]:
# 최초 laboratory value의 pivot table 생성
pivot_table = df5.pivot_table(index=['SUBJECT_ID', 'HADM_ID'], columns='ITEMID', values='VALUENUM', aggfunc='first')
pivot_table = pivot_table.fillna(0)
df_pivot = pivot_table.reset_index(level=['SUBJECT_ID', 'HADM_ID'])
chartevent_value_48h = df_pivot
chartevent_value_48h

ITEMID,SUBJECT_ID,HADM_ID,2,3,4,5,24,25,26,29,...,228381,228382,228394,228395,228396,228409,228410,228411,228412,228444
0,36,165660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,91,121205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,94,140037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101,175533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,111,192123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7540,99897,181057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,2.0,0.0,5.0,5.0,5.0,5.0,0.0
7541,99899,188409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7542,99912,189380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7543,99939,159023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
# 칼럼 별로 관측되지 않은 laboratory value 값(0) 비율 확인 
zero_ratios = (chartevent_value_48h == 0).sum() / len(chartevent_value_48h)
zero_ratios = zero_ratios.to_frame()[2:]
zero_ratios = zero_ratios.reset_index().rename(columns={'index': 'ITEMID', 0:'ZERO_RATIOS'})
zero_ratios = pd.merge(zero_ratios, d_item[["ITEMID", "LABEL"]], on='ITEMID')
zero_ratios = zero_ratios.reindex(columns=["ITEMID", "LABEL", "ZERO_RATIOS"])
zero_ratios = zero_ratios.sort_values(by='ZERO_RATIOS', ascending=True)
zero_ratios

,ITEMID,LABEL,ZERO_RATIOS
1012,226543,Religion,0.468125
647,220602,Chloride (serum),0.481246
1154,227442,Potassium (serum),0.481246
649,220615,Creatinine,0.481378
655,220645,Sodium (serum),0.481511
...,...,...,...
1111,227063,WBCScore_ApacheIV,1.000000
1052,226775,SodiumApacheIIScore,1.000000
1109,227061,Ventilated at any time during ICU Day 1,1.000000
1054,226777,TempApacheIIScore,1.000000


In [64]:
# chartevent_value_48h.to_csv("~/project/MIMIC-III/Data/chartevent_value_48h.csv", index=False)